In [2]:
# AUTHENTICATION TO LOCAL WHOIS DATABASE (POSTGRES)
import os
import psycopg2
import pandas as pd
import numpy as np

pg_user   = os.environ['PG_USER']
pg_pass   = os.environ['PG_PASS']
pg_port   = os.environ['PG_PORT']
pg_dbname = os.environ['PG_DBNAME']

conn = psycopg2.connect(database=pg_dbname,
                        host="0.0.0.0",
                        user=pg_user,
                        password=pg_pass,
                        port=pg_port)

cursor = conn.cursor()

In [5]:
# SELECT THE WHOIS TABLE / COPY MANDATORY TABLE COLUMNS 
import plotly.graph_objects as go
import numpy as np
from ipywidgets import interact
from IPython.display import display

s = "SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';"

list_tables = []
cursor.execute(s)
res = cursor.fetchall()

for t_name_table in res:
    list_tables.append(t_name_table[0])

fig = go.FigureWidget()
fig.add_scatter()

xs=np.linspace(0, 6, 100)
tab=""

fig = go.FigureWidget()
fig.add_scatter()

xs=np.linspace(0, 6, 100)

@interact(Table=list_tables)
def update(Table='blue'):
    with fig.batch_update():
        sql = "SELECT * FROM " + Table + ";"
        df = pd.read_sql_query(sql, conn)
        display(list(df.columns.values))

interactive(children=(Dropdown(description='Table', options=('rf_dom_risk', 'pa_ip', 'rf_ip', 'pa_ipr', 'pa_sv…

In [4]:
# PASTE MANDATORY TABLE COLUMNS 
import ipywidgets as widgets


whois_table = widgets.Text(
    value='',
    placeholder='',
    description='Whois Table',
    style={'description_width': '178px'},
    disabled=False
)
w_w_dom = widgets.Text(
    value='',
    placeholder='',
    description='Domain',
    style={'description_width': '178px'},
    disabled=False
)
w_w_dtc = widgets.Text(
    value='',
    placeholder='',
    description='Whois Date Creation',
    style={'description_width': '178px'},
    disabled=False
)
w_wrn_email = widgets.Text(
    value='',
    placeholder='',
    description='Whois Registrant Email',
    style={'description_width': '178px'},
    disabled=False
)
w_wrn_name = widgets.Text(
    value='',
    placeholder='',
    description='Whois Registrant Name',
    style={'description_width': '178px'},
    disabled=False
)
w_wrn_org = widgets.Text(
    value='',
    placeholder='',
    description='Whois Registrant Organization',
    style={'description_width': '178px'},
    disabled=False
)
w_wrn_country = widgets.Text(
    value='',
    placeholder='',
    description='Whois Registrant Country',
    style={'description_width': '178px'},
    disabled=False
)

widgets.VBox([whois_table,w_w_dom,w_w_dtc,w_wrn_email,w_wrn_name,w_wrn_org,w_wrn_country])


In [13]:
# PLAY THE DASHBOARD / FROM SQL QUERIES TO 
from pandasgui import show
import plotly.express as px

# pie chart  / nb valid dom vs priv vs null
cnt_dom_pp = "select count(pp.domain) from (select " + str(w_w_dom.value) +" from "+ str(whois_table.value) +" where "+str(w_wrn_name.value) +" LIKE '%PRIVACY' and "+ str(w_wrn_org.value) +" LIKE '%PRIVACY') as pp;"
cnt_dom_pn = "select count(pn.domain) from (select " + str(w_w_dom.value) +" from "+ str(whois_table.value) +" where "+str(w_wrn_name.value) +" LIKE '%PRIVACY' and "+ str(w_wrn_org.value) +" is null) as pn;"
cnt_dom_np = "select count(np.domain) from (select " + str(w_w_dom.value) +" from "+ str(whois_table.value) +" where "+str(w_wrn_name.value) +" is null and "+str(w_wrn_org.value)+" LIKE '%PRIVACY') as np;"
cnt_dom_nn = "select count(nn.domain) from (select " + str(w_w_dom.value) +" from "+ str(whois_table.value) +" where "+str(w_wrn_name.value) +" is null and "+str(w_wrn_org.value)+" is null) as nn;"

cnt_dom_px = "select count(domain) from "+ str(whois_table.value) +" where "+str(w_wrn_name.value) +" LIKE '%PRIVACY';"
cnt_dom_xp = "select count(domain) from "+ str(whois_table.value) +" where "+str(w_wrn_org.value)  +" LIKE '%PRIVACY';"
cnt_dom_nx = "select count(domain) from "+ str(whois_table.value) +" where "+str(w_wrn_name.value) +" is null;"
cnt_dom_xn = "select count(domain) from "+ str(whois_table.value) +" where "+str(w_wrn_org.value)  +" is null;"

row1=[]
row2=[]
row3=[]
cursor.execute(cnt_dom_pp)
for rec in cursor:
    row1.append(int(rec[0]))
cursor.execute(cnt_dom_pn)
for rec in cursor:
    row1.append(int(rec[0]))
cursor.execute(cnt_dom_np)
for rec in cursor:
    row2.append(int(rec[0]))
cursor.execute(cnt_dom_nn)
for rec in cursor:
    row2.append(int(rec[0]))
cursor.execute(cnt_dom_px)
for rec in cursor:
    row1.append(int(rec[0]))
cursor.execute(cnt_dom_nx)
for rec in cursor:
    row2.append(int(rec[0]))
cursor.execute(cnt_dom_xp)
for rec in cursor:
    row3.append(int(rec[0]))
cursor.execute(cnt_dom_xn)
for rec in cursor:
    row3.append(int(rec[0]))

dg = pd.DataFrame([row1,row2,row3],index=['name_priv','name_null','total'],columns=['org_priv', 'org_null','total'])

# bar chart / count number of domains per whois registrant(_wrn_) org / email
sql_wrn_emails = \
"select lower(" + str(w_wrn_org.value) + ") as org, lower(" + str(w_wrn_email.value) + ") as mail," \
" count(" + str(w_w_dom.value) + ") as nb_dom" \
" from "  + str(whois_table.value) + \
" where " + str(w_wrn_name.value)  + " NOT LIKE '%PRIVACY' " \
"and "    + str(w_wrn_org.value)   + " NOT LIKE '%PRIVACY' " \
"and "    + str(w_wrn_name.value)  + " NOT LIKE ''" \
"and "    + str(w_wrn_org.value)   + " NOT LIKE '' group by lower(" + str(w_wrn_org.value) + "),lower(" + str(w_wrn_email.value) + ");"

df_sql_wrn_emails_org = pd.read_sql_query(sql_wrn_emails, conn)
df_org = df_sql_wrn_emails_org[df_sql_wrn_emails_org['nb_dom']>1]
fig = px.bar(df_sql_wrn_emails_org, x="mail", y="nb_dom",color="org",title="Nb Doms / Whois Registrant Org / Email")
fig.show()

# bar chart / count number of domains per whois registrant(_wrn_) name / email
sql_wrn_emails = \
"select lower(" + str(w_wrn_name.value) + ") as name, lower(" + str(w_wrn_email.value) + ") as mail," \
" count(" + str(w_w_dom.value) + ") as nb_dom" \
" from "  + str(whois_table.value) + \
" where " + str(w_wrn_name.value)  + " NOT LIKE '%PRIVACY' " \
"and "    + str(w_wrn_org.value)   + " NOT LIKE '%PRIVACY' " \
"and "    + str(w_wrn_name.value)  + " NOT LIKE ''" \
"and "    + str(w_wrn_org.value)   + " NOT LIKE '' group by lower(" + str(w_wrn_name.value) + "),lower(" + str(w_wrn_email.value) + ");"

df_sql_wrn_emails_name = pd.read_sql_query(sql_wrn_emails, conn)
df_name = df_sql_wrn_emails_name[df_sql_wrn_emails_name['nb_dom']>5]
fig = px.bar(df_sql_wrn_emails_name, x="mail", y="nb_dom",color="name",title="Nb Doms / Whois Registrant Name / Email")
fig.show()

# bar chart / count number of domains priv
sql_wrn_priv = \
"select lower(" + str(w_wrn_org.value) + ") as org, lower(" + str(w_wrn_email.value) + ") as mail," \
" count(" + str(w_w_dom.value) + ") as nb_dom" \
" from "  + str(whois_table.value) + \
" where " + str(w_wrn_name.value) + " LIKE '%PRIVACY' " \
"or "     + str(w_wrn_org.value)  + " LIKE '%PRIVACY' " \
"group by lower(" + str(w_wrn_org.value) + "),lower(" + str(w_wrn_email.value) + ");"
df_sql_wrn_priv = pd.read_sql_query(sql_wrn_priv, conn)
df_priv = df_sql_wrn_priv[df_sql_wrn_priv['nb_dom']>1]
fig = px.bar(df_priv, x="mail", y="nb_dom",color="org",title="Nb Doms / Whois Registrant Priv / Email")
fig.show()

dg

# bar chart / services per TLD 

,org_priv,org_null,total
name_priv,64,0,272.0
name_null,0,0,0.0
total,65,0,NaN


In [ ]:
conn.close()